# Feature Extraction for Phoneme Recognition on TIMIT

## Goals

- Loading and testing the datasets exported by the previous notebook.
- Feature design for the LSTM-250 network.
- Feature extraction from the TIMIT dataset.
- Exporting the features as standalone dataset for training the network.
    - Feature dataset record: (phoneme, feature-sequence)

# Environment Setup

In [ ]:
import torch

import os
import librosa
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd
from tqdm.auto import tqdm


if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print('Using PyTorch version:', torch.__version__, ' Device:', device)

# Loading and Testing the Datasets

In [ ]:
ds_path = './session/curated-dataset.pt'
ds_dict = torch.load(ds_path)
print(ds_dict.keys())
print(ds_dict['note'])

Train_ds = ds_dict['train']
Test_ds  = ds_dict['test']

print('Train_ds:', len(Train_ds))
ipd.display(Train_ds[:3])
print('Test_ds:', len(Test_ds))
ipd.display(Test_ds[:3])

In [ ]:
rec = Train_ds[1]
phone, audio_path, start, end = rec
wave, rate = librosa.load(audio_path, sr=None)
phone_wave = wave[start:end]
print(phone)
ipd.display(ipd.Audio(phone_wave, rate=rate))

In [ ]:
# delete redundant variables to avoid confusion
del ds_path, ds_dict
del rec, phone, audio_path, start, end, wave, rate, phone_wave
print(dir())

## Convert to Audio dataset

Convert records from (phoneme, path-to-audio, start-index, end-index) to (phoneme, wave, rate).

In [ ]:
# Given a file path, returns the audio waveform and the sampling rate.
# Mainly used for caching already loaded files.
audio_cache = {}    # Caching loaded audio files for faster processing
def getAudio(audio_path):
    if audio_path not in audio_cache:
        wave, rate = librosa.load(audio_path, sr=None)    
        audio_cache[audio_path] = (wave, rate)
    return audio_cache[audio_path]


# Given file path, start, and end indices, returns the audio slice and the sampling rate
def getAudioSlice(audio_path, start, end):
    wave, rate = getAudio(audio_path)
    return wave[start:end], rate


# Without caching: 14 seconds
# With caching: < 1sec
# Test above function
for rec in tqdm(Train_ds):    
    _, audio_path, *_ = rec
    getAudio(audio_path)
    
    
# delete redundant variables to avoid confusion
del rec, audio_path

In [ ]:
# Given a phoneme record list, returns an audio record list: (phoneme, wave, rate)
def makeAudioDS(list_phone_rec):
    audio_ds = []
    for phone_rec in list_phone_rec:
        phone, audio_path, start, end = phone_rec
        wave, rate = getAudioSlice(audio_path, start, end)
        audio_rec = [phone, wave, rate]
        audio_ds.append(audio_rec)
    return audio_ds


# Convert to audio datasets
Train_audio_ds = makeAudioDS(Train_ds)
Test_audio_ds  = makeAudioDS(Test_ds)
        
print('Train_audio_ds:', len(Train_audio_ds))
print('Test_audio_ds :', len(Test_audio_ds))

# Feature Design

**NOTE:**
- Feature vector is designed following [[Speech-Recog paper]](https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=6638947)
- Mel-spectrogram is used as the base feature, 40 of Mel bands are generated.
- The energy term is computed using the mel-spectrogram.
- First and second order derivatives of those terms are used.
- Total length of the feature vector: 41 x 3 = 123.

In [ ]:
# Given an audio waveform, get the MFCC coefficients
# Params,
#   n_mfcc    : no. of MFCC coefficient to return
#   n_mels    : number of Mel bands to generate
#   fft_window: length of the FFT window
#   hop_len   : number of audio samples between adjacent STFT columns.
# Note: Change fft_window and hop_len to play with the sequence lengths.
#def getMFCC(wave, sample_rate, n_mfcc, fft_window, hop_length, n_mels):
#    #mfcc = librosa.feature.mfcc(y=wave, sr=sample_rate, n_mfcc=n_mfcc, n_fft=fft_window, hop_length=hop_length, n_mels=n_mels)
#    mel_spec = librosa.feature.melspectrogram(y=wave, sr=sample_rate, n_fft=fft_window, hop_length=hop_length, n_mels=n_mels)
#    log_mel_spec = librosa.power_to_db(mel_spec)  # Convert to log-scale
#    return log_mel_spec
    #return mfcc


# Given a sequence of mfcc coefficients, returns a sequence of corresponding energy terms
#def getEnergy(mfcc_seq):
#    energy_seq = np.sum(mfcc_seq**2, axis=0)
#    return energy_seq


# Given an audio waveform, get the mel-spectrogram
# Params,
#   n_mels     : number of Mel bands to generate
#   fft_window : length of the FFT window
#   hop_length : number of audio samples between adjacent STFT columns.
# Note: Change fft_window and hop_len to play with the sequence lengths.
def getMelSpec(wave, sample_rate, n_mels, fft_window, hop_length):
    mel_spec = librosa.feature.melspectrogram(y=wave, sr=sample_rate, n_fft=fft_window, hop_length=hop_length, n_mels=n_mels)
    log_mel_spec = librosa.power_to_db(mel_spec)  # Convert to log-scale
    return log_mel_spec


# Given a waveform, compute energy in each frame
def getEnergy(wave, frame_length, hop_length):
    energy = librosa.feature.rms(y=wave, frame_length=frame_length, hop_length=hop_length)
    return energy


# Given a sequence, computes the derivative of order=order.
def getDelta(seq, order):
    delta = librosa.feature.delta(seq, order=order)
    return delta


# Plotting utilities ---
def plotAudio(wave, sample_rate, axis):
    duration = len(wave) / sample_rate
    time = np.linspace(0, duration, len(wave))
    axis.plot(time, wave)
    

# Use librosa.display.specshow to display 2D features
def plotSpecshow(data, fig, axis):
    img = librosa.display.specshow(data, ax=axis)
    fig.colorbar(img, ax=axis)
    return img
    

In [ ]:
# Test above functions ---
# Get an audio
rec = Test_audio_ds[1200]
phone, wave, rate = rec
print('phone:', phone, '  wave:', len(wave), '  rate:', rate)

#mfcc = getMFCC(wave, rate, n_mfcc=40, fft_window=64, hop_length=16, n_mels=40)
#print('mfcc:', mfcc.shape)

mel_spec = getMelSpec(wave, rate, n_mels=40, fft_window=64, hop_length=16)
print('mel_spec:', mel_spec.shape)

#energy = getEnergy(mfcc)
energy = getEnergy(wave, frame_length=64, hop_length=16)
print('energy:', energy.shape)

#mfcc_eng = np.vstack([mfcc, energy])
#print('mfcc_eng:', mfcc_eng.shape)

mel_eng = np.vstack([mel_spec, energy])

delta1 = getDelta(mel_eng, order=1)
delta2 = getDelta(mel_eng, order=2)
print('delta1:', delta1.shape)
print('delta2:', delta2.shape)

In [ ]:
# Plot the features
print('phoneme:', phone)
ipd.display(ipd.Audio(wave, rate=rate))

fig = plt.figure(figsize=(3*4, 7))
fig.subplots_adjust(hspace=0.5)
fig.tight_layout()

ax_wave = fig.add_subplot(3, 1, 1)
plotAudio(wave, rate, ax_wave)
ax_wave.set_title('Audio')

ax_mel = fig.add_subplot(3, 2, 3)
ax_mel.set_title('Mel Spectrogram')
plotSpecshow(mel_spec, fig, ax_mel)

ax_energy = fig.add_subplot(3, 2, 4)
ax_energy.set_title('Energy')
ax_energy.plot(energy[0], color='r')

ax_delta1 = fig.add_subplot(3, 2, 5)
ax_delta1.set_title('Delta-1')
plotSpecshow(delta1, fig, ax_delta1)

ax_delta2 = fig.add_subplot(3, 2, 6)
ax_delta2.set_title('Delta-2')
plotSpecshow(delta2, fig, ax_delta2)

In [ ]:
# delete redundant variables to avoid confusion
del rec, phone, wave, rate
del mel_spec, energy, mel_eng, delta1, delta2
del ax_wave, ax_mel, ax_energy, ax_delta1, ax_delta2, fig

## Check Audio Dataset Distribution

Check the audio dataset to determine the parameters for the features.  

In [ ]:
from math import inf as INF


# Given a audio dataset, returns the record with min and max audio lengths
def getMinMaxRec(ds_list):
    min_len = INF
    max_len = -INF
    min_rec = None
    max_rec = None
    all_len = []
    for audio_rec in ds_list:    # rec: (phoneme, wave, rate)
        phone_len = len(audio_rec[1])
        if phone_len < min_len:
            min_len = phone_len
            min_rec = audio_rec
        if phone_len > max_len:
            max_len = phone_len
            max_rec = audio_rec
        all_len.append(phone_len)
    return min_len, max_len, min_rec, max_rec, all_len


# Given an audio dataset, shows the audio length disribution
def showAudioLenDisrib(audio_ds):
    # Print min/max info
    min_len, max_len, min_rec, max_rec, all_len = getMinMaxRec(audio_ds)
    print('min_rec:', min_len, min_rec[0])
    print('max_rec:', max_len, max_rec[0])
    median = np.median(all_len)
    print('median:', median)
    
    # show min-rec waveform
    wave, rate = min_rec[1:]
    print('min_rec waveform:')
    plt.plot(wave)
    plt.show()
    
    # Play the audio clips
    ipd.display(ipd.Audio(wave, rate=rate))
    wave, rate = max_rec[1:]
    ipd.display(ipd.Audio(wave, rate=rate))
    
    # Show the histogram
    plt.hist(all_len, bins='auto', rwidth=0.8)
    plt.xlabel('Value')
    plt.ylabel('Frequency')
    plt.title('Distribution of audio lengths')

In [ ]:
print('Train_audio_ds ---')
showAudioLenDisrib(Train_audio_ds)

In [ ]:
print('Test_audio_ds ---')
showAudioLenDisrib(Test_audio_ds)

## Zero Padding of Short Audio

**NOTE:**
Based on the above observation
- Because median is around 1024, pad zeros to make all audio length >= 1024.

In [ ]:
# Given an audio dataset, pad zeros to make all audio clips lengths >= min_len
# audio_ds[i]: (phoneme, wave, rate)
def padZeroAudio(audio_ds, min_len):
    for audio_rec in audio_ds:
        current_length = len(audio_rec[1])
        if current_length < min_len:
            num_zeros = min_len - current_length
            padding = np.zeros(num_zeros)
            padded_audio = np.concatenate((audio_rec[1], padding))
            audio_rec[1] = padded_audio

            
# Pad zeros to audio
F_min_audio_len = 1024
padZeroAudio(Train_audio_ds, F_min_audio_len)
padZeroAudio(Test_audio_ds, F_min_audio_len)    

In [ ]:
print('Train_audio_ds ---')
showAudioLenDisrib(Train_audio_ds)

In [ ]:
print('Test_audio_ds ---')
showAudioLenDisrib(Test_audio_ds)

# Feature Extraction and Export

In [ ]:
F_fft_window = 256
F_hop_length = 64
F_n_mels = 40

F_note = f'''
F_n_mels    : {F_n_mels}
F_fft_window: {F_fft_window}
F_hop_length: {F_hop_length}
'''

# Given an audio record, returns a feature record: (phoneme, feature-sequence)
def getFeatureRecord(audio_record):
    # Feature extraction parameters
    # Extract features from the audio-record
    phone, wave, rate = audio_record
    # compute mel-spectrogram
    mel_spec = getMelSpec(wave, rate, n_mels=F_n_mels, fft_window=F_fft_window, hop_length=F_hop_length)
    # Compute energy from mfcc then stack on top of mfcc for deta calculation
    energy = getEnergy(wave, frame_length=F_fft_window, hop_length=F_hop_length)
    mel_eng = np.vstack([mel_spec, energy])
    # compute deltas
    delta1 = getDelta(mel_eng, order=1)
    delta2 = getDelta(mel_eng, order=2)
    # stack all to make feature vector
    feat_vec = np.vstack([mel_eng, delta1, delta2])
    return [phone, feat_vec]    # make each record a list, not a tuple

    
# Test above function
audio_rec = Train_audio_ds[4]
phone, feat_vec = getFeatureRecord(audio_rec)
print('phone:', phone)
print('audio_rec[1]:', len(audio_rec[1]))
print('feat_vec:', feat_vec.shape)
F_feat_len = len(feat_vec)
ipd.display(ipd.Audio(audio_rec[1], rate=audio_rec[2]))


# delete redundant variables to avoid confusion
del audio_rec, phone, feat_vec

In [ ]:
# Given an audio dataset, returns a list of feature dataset
# audio_ds[i]: (phone, wave, rate)
# return[i]: (phone, feature-sequence)
def makeFeatureDS(audio_ds):
    feat_ds = []
    for audio_rec in tqdm(audio_ds):
        feat_rec = getFeatureRecord(audio_rec)
        feat_ds.append(feat_rec)
    return feat_ds


# Given an audio-dataset, converts it into feature-dataset and saves in a file
def saveFeatureDS(audio_ds, save_path, note):
    feat_ds = makeFeatureDS(audio_ds)
    assert len(feat_ds) == len(audio_ds), "Test dataset length mismatch"
    export = {
        'note' : note,
        'data-schema' : '(phoneme, feature-sequence)',
        'data' : feat_ds
    }
    torch.save(export, save_path)
    print(f'INFO: Saved {save_path}')
    

In [ ]:
# Export the dataset with necessary information for the next notebook
note = f'''
Notes:
- Feature record: (phone, feature_sequence)
- feature_seqence: list(feature_vector)
- len(feature_vector): {F_feat_len}

Features are extracted using following parameters''' + F_note

print(note)

In [ ]:
# Make test feature dataset then save
saveFeatureDS(Test_audio_ds, './session/test-features.pt', note)

In [ ]:
saveFeatureDS(Train_audio_ds, './session/train-features.pt', note)